In [1]:
import os
import pandas as pd
import numpy as np

from utils import utils

In [3]:
data_model = utils.load_and_backup_dm("../EL.data.model.csv", "../backups/")

In [ ]:
# Updates 2023-09-21
dm.loc[dm["Attribute"] == "TRUE", "DependsOn"] = np.nan
# clean up source column
dm["Source"] = (
    dm["Source"]
    .fillna("")
    .str.split(",")
    .apply(lambda x: ",".join(sorted(np.unique(x))))
)
dm["Valid Values"] = (
    dm["Valid Values"]
    .fillna("")
    .str.split(",")
    .apply(lambda x: ",".join([y.strip() for y in x]))
)
dm["DependsOn"] = (
    dm["DependsOn"]
    .fillna("")
    .str.split(",")
    .apply(lambda x: ",".join([y.strip() for y in x]))
)
dm.to_csv("../EL.data.model.csv")

In [15]:
import frontmatter
import re
from mdutils import fileutils

data_model.rename(
    {"Attribute": "Key", "Description": "Key Description"}, axis=1, inplace=True
)

data_model = data_model[["Key", "Key Description", "Type", "Source", "Module"]]

data_model.to_csv("../_data/FullTable.csv", index=False)

post = frontmatter.load("../template_page_template.md")
post.metadata["title"] = re.sub("([A-Z]+)|_", r" \1", "FullTable").title()
post.metadata["permalink"] = f'docs/{post.metadata["title"]}.html'
post.metadata["parent"] = "Full Table"
post.content = (
    "{% assign mydata=site.data."
    + "fullTable"
    + " %} \n{: .note-title } \n"
    + f">{post.metadata['title']}\n"
    + ">\n"
    + f">Full Table of Terms ELITE portal\n"
    + post.content
)

# create directory for the moduel if not exist
if not os.path.exists(f"../docs/Full Table/"):
    os.mkdir(f"../docs/Full Table/")

    # create a module page
    module = fileutils.MarkDownFile(f"../docs/FullTable/FullTable")

    module.append_end(
        f"--- \nlayout: page \ntitle: Full Table \nhas_children: true \nnav_order: 2 \npermalink: FullTable.html \n---"
    )
# create file
file = fileutils.MarkDownFile(f"../docs/FullTable/FullTable")
# add content to the file
file.append_end(frontmatter.dumps(post))